The with statement sets up a temporary context and reliably tears it down, under the control of a context manager object. This prevents errors and reduces boilerplate code, making APIs at the same time safer and easier to use.

## Do This, Then That: else Blocks Beyond if

#### Here are the rules:

**for** : The else block will run only if and when the for loop runs to completion (i.e., not if the for is aborted with a break).

**while** : The else block will run only if and when the while loop exits because the condition became falsy (i.e., not when the while is aborted with a break).

**try** : The else block will only run if no exception is raised in the try block. The official docs also state: “Exceptions in the else clause are not handled by the preceding except clauses.”

*I think else is a very poor choice for the keyword in all cases except if. It implies an excluding alternative, like “Run this loop, otherwise do that,” but the semantics for else in loops is the opposite: “**Run this loop, then do that**.” This suggests then as a better keyword—which would also make sense in the try context: “**Try this, then do that**.”*

In [1]:
my_list = ['apple', 'banana', 'mango']
for item in my_list:
    if item == 'banana':
        print('Yay!')
        break
else:
    raise ValueError('No banana flavor found!')

Yay!


In the case of try/except blocks, else may seem redundant at first. After all, the after_call() in the following snippet will run only if the dangerous_call() does not raise an exception, correct?

In [2]:
def dangerous_call():
    print("Danger!")

In [3]:
def after_call():
    print("After danger!")

In [4]:
try:
    dangerous_call()
    after_call()
except OSError:
    log('OSError...')

Danger!
After danger!


However, doing so puts the after_call() inside the try block for no good reason. *For clarity and correctness, the body of a try block should only have the statements that may generate the expected exceptions*. This is much better:

In [5]:
try:
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()

Danger!
After danger!


***In Python, try/except is commonly used for control flow, and not just for error handling***. There’s even an acronym/slogan for that documented in the official Python glossary:

**EAFP**

***Easier to ask for forgiveness than permission***. This common Python coding style assumes the existence of valid keys or attributes and catches exceptions if the assumption proves false. This clean and fast style is characterized by the presence of many try and except statements. The technique contrasts with the LBYL style common to many other languages such as C.

The glossary then defines LBYL:

**LBYL**

***Look before you leap***. This coding style explicitly tests for pre-conditions before making calls or lookups. This style contrasts with the EAFP approach and is characterized by the presence of many if statements. In a multi-threaded environment, the LBYL approach can risk introducing a race condition between “the looking” and “the leaping”. For example, the code, if key in mapping: return mapping[key] can fail if another thread removes key from mapping after the test, but before the lookup. This issue can be solved with locks or by using the EAFP approach.

## Context Managers and with Blocks

Context manager objects exist to control a with statement, just like iterators exist to control a for statement.

The with statement was designed to simplify the try/finally pattern, which guarantees that some operation is performed after a block of code, even if the block is aborted because of an exception, a return or sys.exit() call. The code in the finally clause usually releases a critical resource or restores some previous state that was temporarily changed.

The context manager protocol consists of the `__enter__` and `__exit__` methods. At the start of the with, `__enter__` is invoked on the context manager object. The role of the finally clause is played by a call to `__exit__` on the context manager object at the end of the with block.

In [6]:
with open('vector.py') as fp:
    src = fp.read(60)

In [7]:
len(src)

60

In [8]:
fp

<_io.TextIOWrapper name='vector.py' mode='r' encoding='cp1252'>

In [9]:
try:
    fp.read(60)
except ValueError as err:
    print(f"{type(err).__name__}: {str(err)}")

ValueError: I/O operation on closed file.


The `open()` function returns an instance of TextIOWrapper, and its `__enter__` method returns `self`. But the `__enter__` method may also return some other object instead of the context manager.

When control flow exits the with block in any way, the `__exit__` method is invoked on the context manager object, not on whatever is returned by `__enter__`.

The as clause of the with statement is optional. In the case of open, you’ll always need it to get a reference to the file, but some context managers return None because they have no useful object to give back to the user.

In [10]:
class LookingGlass:
    
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
    
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print("Please DO NOT divide by zero!")
            return True

If `__exit__` returns `None` or anything but `True`, any exception raised in the with block will be propagated.

In [11]:
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [12]:
what
# Now the with block is over. We can see that the value returned by __enter__,
# held in what, is the string 'JABBERWOCKY'.

'JABBERWOCKY'

In [13]:
print("Back to normal")

Back to normal


Context managers are a fairly novel feature and slowly but surely the Python community is finding new, creative uses for them. Some examples from the standard library are:
* Managing transactions in the sqlite3 module.
* Holding locks, conditions, and semaphores in threading code.
* Setting up environments for arithmetic operations with Decimal objects; see the decimal.localcontext documentation.
* Applying temporary patches to objects for testing; see the <a href="http://bit.ly/1MM8imk">unittest.mock.patch function</a>.

## Using @contextmanager

The @contextmanager decorator reduces the boilerplate of creating a context manager: instead of writing a whole class with `__enter__`/`__exit__` methods, you just implement a generator with a single yield that should produce whatever you want the `__enter__` method to return.

In a generator decorated with @contextmanager, yield is used to split the body of the function in two parts: everything before the yield will be executed at the beginning of the while block when the interpreter calls `__enter__`; the code after yield will run when `__exit__` is called at the end of the block.

In [14]:
import contextlib

In [15]:
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write 
    
    def reverse_write(text): 
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write 
    yield 'JABBERWOCKY'

    sys.stdout.write = original_write

In [16]:
with looking_glass() as what: 
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [17]:
print(what)

JABBERWOCKY


Essentially the contextlib.contextmanager decorator wraps the function in a class that implements the `__enter__` and `__exit__` methods.

The `__enter__` method of that class:

1. Invokes the generator function and holds on to the generator object—let’s call it gen.

2. Calls next(gen) to make it run to the yield keyword.

3. Returns the value yielded by next(gen), so it can be bound to a target variable in the with/as form.

When the with block terminates, the `__exit__` method:

1. Checks an exception was passed as exc_type; if so, gen.throw(exception) is invoked, causing the exception to be raised in the yield line inside the generator function body.

2. Otherwise, next(gen) is called, resuming the execution of the generator function body after the yield.

Example 15-5 has a serious flaw: if an exception is raised in the body of the with block, the Python interpreter will catch it and raise it again in the yield expression inside looking_glass. But there is no error handling there, so the looking_glass function will abort without ever restoring the original sys.stdout.write method, leaving the system in an invalid state.

In [18]:
@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write 
    
    def reverse_write(text): 
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write 
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except Exception as err:
        msg = f'{type(err).__name__}: {str(err)}'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

Recall that the `__exit__` method tells the interpreter that it has handled the exception by returning True; in that case, the interpreter suppresses the exception. On the other hand, if `__exit__` does not explicitly return a value, the interpreter gets the usual `None`, and propagates the exception. With `@contextmanager`, the default behavior is inverted: the `__exit__` method provided by the decorator assumes any exception sent into the generator is handled and should be suppressed. You must explicitly re-raise an exception in the decorated function if you don’t want @contextmanager to suppress it.

Having a try/finally (or a with block) around the yield is an unavoidable price of using @contextmanager, because you never knowwhat the users of your context manager are going to do inside their with block.

## Use Case: Rewriting Files in-place

In [19]:
import io
import os
import csv

In [20]:
@contextlib.contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.

    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'):
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                       encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                           encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

In [21]:
# with inplace(csvfilename, 'r', newline='') as (infh, outfh):
#     reader = csv.reader(infh)
#     writer = csv.writer(outfh)

#     for row in reader:
#         row += ['new', 'columns']
#         writer.writerow(row)